<a href="https://colab.research.google.com/github/Mario-td/Hand-Gesture-Classification-with-Tensorflow-2.0/blob/master/GatheringData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Gathering data from text files**

In [11]:
import pandas as pd
import numpy as np

In [ ]:
# Allows access to Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [13]:
# Functions for loading the data and labels
def load_data(path, steps):
    file = open(path, 'r')
    data = []
    for line in file.readlines():
        data.append([float(i) for i in line.split(',') if i.strip()])
    file.close()

    # Subdivide the list by the number of steps per sample
    data = [data[x:x+steps] for x in range(0, len(data), steps)]
    return data


def load_label(path):
    file = open(path, 'r')
    label = []
    for line in file.readlines():
        for i in line.split():
            # -1 for indexing from 0
            l = int(i) - 1
        label.append(l)
    file.close()
    return label


# Function for naming the columns
def feat3D(n_features3D):
    feat = []
    for i in range(0, n_features3D):
        feat.append('j%d_x' % i)
        feat.append('j%d_y' % i)
        feat.append('j%d_z' % i)
    return feat

In [14]:
# Number of steps per sample
n_steps = 32
# Number of keypoint per step
n_keypoints = 21

# Path to the files
path = '/content/drive/My Drive/MastersThesis/Dataset/'

# Loads data and labels
X_train3D = load_data(path + 'X_train3D.txt', n_steps)
Y_train = load_label(path + 'Y_train.txt')

X_test3D = load_data(path + 'X_test3D.txt', n_steps)
Y_test = load_label(path + 'Y_test.txt')

In [15]:
# Visualizes the shape of the arrays
print(np.array(X_train3D).shape)
print(np.array(Y_train).shape)
print(np.array(X_test3D).shape)
print(np.array(Y_test).shape)

(3606, 32, 63)
(3606,)
(362, 32, 63)
(362,)


# **Structures the dataset**

In [16]:
# Sets the labels as index and the coordinate arrays as columns
df_train = pd.DataFrame(X_train3D, index=Y_train)
# MultiIndex with labels and sequences
df_train = df_train.apply(pd.Series).stack()
df_train = pd.DataFrame(df_train, columns=['3D position'])
df_train = df_train.rename_axis(['Label', 'Sequence'])
df_train

3D position
Label Sequence                                                   
0     0         [273.65625, 375.640625, 0.0, 312.046875, 326.2...
      1         [241.234375, 374.203125, 0.0, 284.609375, 319....
      2         [233.46875, 369.3125, 0.0, 266.28125, 320.0937...
      3         [223.5, 358.78125, 0.0, 256.03125, 315.40625, ...
      4         [219.0, 360.0, 0.0, 252.0, 310.5, 257.0, 274.0...
...                                                           ...
4     27        [380.5625, 336.59375, 1542.0, 406.3125, 330.15...
      28        [387.21875, 344.625, 1542.0, 406.125, 329.5, 1...
      29        [385.5625, 343.375, 1542.0, 404.15625, 335.937...
      30        [385.03125, 344.1875, 1542.0, 403.703125, 332....
      31        [381.9375, 346.4375, 1542.0, 400.375, 335.375,...

[115392 rows x 1 columns]

In [17]:
# Names the new columns and splits them
df_train[feat3D(n_keypoints)] = pd.DataFrame(df_train['3D position'].tolist(),
                                             index=df_train.index)
# Delete the first column
del df_train['3D position']
df_train

j0_x        j0_y    j0_z  ...       j20_x       j20_y   j20_z
Label Sequence                                  ...                                
0     0         273.656250  375.640625     0.0  ...  196.875000  200.140625     0.0
      1         241.234375  374.203125     0.0  ...  132.796875  211.546875     0.0
      2         233.468750  369.312500     0.0  ...  113.156250  210.718750     0.0
      3         223.500000  358.781250     0.0  ...  104.218750  217.812500     0.0
      4         219.000000  360.000000     0.0  ...   87.000000  217.000000     0.0
...                    ...         ...     ...  ...         ...         ...     ...
4     27        380.562500  336.593750  1542.0  ...  332.281250  269.000000  1542.0
      28        387.218750  344.625000  1542.0  ...  315.375000  253.875000  1799.0
      29        385.562500  343.375000  1542.0  ...  311.187500  250.406250  1542.0
      30        385.031250  344.187500  1542.0  ...  306.609375  247.093750  1542.0
      31        381.937500  346.437500  1542.0  ...  308.187500  246.875000  1542.0

[115392 rows x 63 columns]

In [18]:
# Same procedure for the test set
# Sets the labels as index and the coordinate arrays as columns
df_test = pd.DataFrame(X_test3D, index=Y_test)
# MultiIndex with labels and sequences
df_test = df_test.apply(pd.Series).stack()
df_test = pd.DataFrame(df_test, columns=['3D position'])
df_test = df_test.rename_axis(['Label', 'Sequence'])
# Names the new columns and splits them
df_test[feat3D(n_keypoints)] = pd.DataFrame(df_test['3D position'].tolist(),
                                            index=df_test.index)
# Delete the first column
del df_test['3D position']
df_test

j0_x        j0_y    j0_z  ...       j20_x       j20_y   j20_z
Label Sequence                                 ...                                
0     0         367.15625  323.156250   771.0  ...  314.187500  185.437500   771.0
      1         336.78125  349.593750   771.0  ...  347.703125  214.890625   771.0
      2         334.25000  350.250000   771.0  ...  341.750000  219.000000   771.0
      3         359.62500  326.625000   771.0  ...  386.625000  205.125000   771.0
      4         393.46875  320.218750   771.0  ...    0.000000    0.000000     0.0
...                   ...         ...     ...  ...         ...         ...     ...
4     27        217.15625  319.265625  1286.0  ...  151.296875  178.765625  1543.0
      28        214.56250  300.296875  1799.0  ...  174.546875  201.796875  2056.0
      29        219.43750  304.921875  1799.0  ...  216.765625  259.500000     0.0
      30        216.90625  313.468750     0.0  ...  216.906250  259.000000  1542.0
      31        227.43750  317.437500     0.0  ...  214.156250  261.656250  2056.0

[11584 rows x 63 columns]

In [19]:
# Saves the dataset in CSV format
df_train.to_csv(path + '/TrainingSet3D.csv')
df_test.to_csv(path + '/TestSet3D.csv')
